# ISTAT

In [1]:
import sqlite3
conn = sqlite3.connect("D:/files/istat.sqlite")
cursor = conn.cursor()
import requests
import xml.etree.ElementTree as ET
import pandas as pd

root = 'https://esploradati.istat.it/SDMXWS/rest/data/IT1,'
# https://git.nomics.world/dbnomics-source-data/istat-source-data

## CONTI NAZIONALI - DATAWAREHOUSE ISTAT

### 92_506 - Prodotto interno lordo e principali componenti 	
- web         https://esploradati.istat.it/databrowser/#/it/dw/categories/IT1,DATAWAREHOUSE,1.0/UP_ACC_ANNUAL/IT1,92_506_DF_DCCN_PILN_1,1.0
- struttura   https://esploradati.istat.it/SDMXWS/rest/dataflow/IT1/92_506_DF_DCCN_PILN_1/1.0/?detail=Full&references=Descendants
- dato        https://esploradati.istat.it/SDMXWS/rest/data/IT1,92_506_DF_DCCN_PILN_1,1.0/A...../ALL/?detail=full&startPeriod=2015-01-01&endPeriod=2024-12-31&dimensionAtObservation=TIME_PERIOD

In [7]:
CL_REFAREA = pd.read_csv('../dati/CL_REFAREA.csv', sep='\t')
CL_REFAREA.sample()

,area,Istat,NUT
3934,Vigodarzere,028099,ITD36


In [3]:
# https://git.nomics.world/dbnomics-source-data/istat-source-data/-/blob/60f8c3d948490282df509d21fcf8bd33b92d7920/codelist/CL_TIPO_DATO21.xml
data = { "Codice": ["GDPXIN", "GDPXFTE", "FDCXIN", "FNCXIN", "CEXFTEE", "WSXFTEE", "GNIXIN", "B1GQ_B_W2_S1_R_POP", "B1GQ_B_W2_S1_R_FT",
                     "B1GQ_B_W2_S1_R_PS", "B1GQ_B_W2_S1_R_HW", "B1G_B_W2_S1_R_FT", "B1G_B_W2_S1_R_PS", "B1G_B_W2_S1_R_HW","P3_D_W2_S1_R_POP", 
                     "P3_D_W0_S1_R_POP",  "D1_D_W2_S1_R_FT", "D1_D_W2_S1_R_PS", "D1_D_W2_S1_R_HW",   "D11_D_W2_S1_R_FT", "D11_D_W2_S1_R_PS", 
                     "D11_D_W2_S1_R_HW", "B5GQ_B_W0_S1_R_POP","B1G_B_W2_S1_R_POP","B6G_B_W2_S14A_R_POP"],

                     
    "Descrizione": ["prodotto interno lordo ai prezzi di mercato per abitante",  "prodotto interno lordo ai prezzi di mercato per unità di lavoro totali", "consumi finali interni per abitante",  "consumi finali nazionali per abitante", "redditi da lavoro dipendente per unità di lavoro dipendente",
    "retribuzioni lorde per unità di lavoro dipendente",  "reddito nazionale lordo ai prezzi di mercato per abitante", "prodotto interno lordo ai prezzi di mercato per abitante",
      "prodotto interno lordo ai prezzi di mercato per unità di lavoro totali",
    "prodotto interno lordo ai prezzi di mercato per occupato",  "prodotto interno lordo ai prezzi di mercato per ora lavorata", "valore aggiunto per unità di lavoro",
      "valore aggiunto per occupato", "valore aggiunto per ora lavorata",  "consumi finali interni per abitante",
    "consumi finali nazionali per abitante", "redditi interni da lavoro dipendente per unità di lavoro dipendente", "redditi interni da lavoro dipendente per occupato","redditi interni da lavoro dipendente per ora lavorata",
    "retribuzioni interne lorde per unità di lavoro dipendente",  "retribuzioni interne lorde per occupato",  "retribuzioni interne lorde per ora lavorata",  
    "reddito nazionale lordo ai prezzi di mercato per abitante","valore aggiunto per abitante","reddito disponibile delle famiglie consumarici per abitante"]}
TYPE_AGGR = pd.DataFrame(data)

### CONTI E AGGREGATI ECONOMICI NAZIONALI ANNUALI
- https://esploradati.istat.it/databrowser/#/it/dw/categories/IT1,DATAWAREHOUSE,1.0/UP_ACC_ANNUAL/IT1,92_1225_DF_DCCN_ANA1_7,1.0
- https://esploradati.istat.it/SDMXWS/rest/dataflow/IT1/92_1225_DF_DCCN_ANA1_7/1.0/?detail=Full&references=Descendants
- https://esploradati.istat.it/SDMXWS/rest/data/IT1,92_1225_DF_DCCN_ANA1_7,1.0/A........./ALL/?detail=full&startPeriod=2015-01-01&endPeriod=2024-12-31&dimensionAtObservation=TIME_PERIOD

#### 92_1225

In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

In [4]:
dataset = '92_1225_DF_DCCN_ANA1_7' # nazionale
data_url = f'https://esploradati.istat.it/SDMXWS/rest/data/IT1,{dataset},1.0/A........./ALL/?detail=full&startPeriod=2015-01-01&endPeriod=2024-12-31&dimensionAtObservation=TIME_PERIOD'
response = requests.get(data_url)
root = ET.fromstring(response.content)
# 🔎 Namespace XML per SDMX aggiornato
ns = {"message": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message","generic": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic"}
data = []
for series in root.findall(".//generic:Series", namespaces=ns):
    series_key = series.find("generic:SeriesKey", namespaces=ns)
    attributes = {value.attrib["id"]: value.attrib["value"] for value in series_key.findall("generic:Value", namespaces=ns)}
    for obs in series.findall("generic:Obs", namespaces=ns):
        time_period = obs.find("generic:ObsDimension", namespaces=ns).attrib["value"]
        obs_value = obs.find("generic:ObsValue", namespaces=ns).attrib["value"]
        data.append({**attributes, "TIME_PERIOD": time_period, "OBS_VALUE": float(obs_value)})
dati = pd.DataFrame(data)
dati = dati.query('VALUATION == "V"')
dati = dati[['DATA_TYPE_AGGR','EDITION', 'TIME_PERIOD', 'OBS_VALUE']]
dati = pd.merge(dati, TYPE_AGGR, how ='left', left_on = 'DATA_TYPE_AGGR', right_on = 'Codice')
dati = dati[['Descrizione','Codice', 'EDITION','TIME_PERIOD','OBS_VALUE']]
dati.to_sql(dataset, conn, if_exists="replace", index=False)
dati.sort_values(by =['Descrizione','EDITION','TIME_PERIOD'], ascending=[True, False, False])

,Descrizione,Codice,EDITION,TIME_PERIOD,OBS_VALUE
265,consumi finali nazionali per abitante,P3_D_W0_S1_R_POP,2025M3,2024,28014.991
264,consumi finali nazionali per abitante,P3_D_W0_S1_R_POP,2025M3,2023,27334.003
263,consumi finali nazionali per abitante,P3_D_W0_S1_R_POP,2025M3,2022,26145.138
262,consumi finali nazionali per abitante,P3_D_W0_S1_R_POP,2025M3,2021,23667.329
261,consumi finali nazionali per abitante,P3_D_W0_S1_R_POP,2025M3,2020,22080.609
...,...,...,...,...,...
4,valore aggiunto per unità di lavoro,B1G_B_W2_S1_R_FT,2024M9,2019,67534.836
3,valore aggiunto per unità di lavoro,B1G_B_W2_S1_R_FT,2024M9,2018,66776.103
2,valore aggiunto per unità di lavoro,B1G_B_W2_S1_R_FT,2024M9,2017,66069.996
1,valore aggiunto per unità di lavoro,B1G_B_W2_S1_R_FT,2024M9,2016,65124.268


In [23]:
dati['Descrizione'].unique()

array(['valore aggiunto per unità di lavoro',
       'valore aggiunto per ora lavorata', 'valore aggiunto per occupato',
       'prodotto interno lordo ai prezzi di mercato per unità di lavoro totali',
       'prodotto interno lordo ai prezzi di mercato per ora lavorata',
       'prodotto interno lordo ai prezzi di mercato per abitante',
       'prodotto interno lordo ai prezzi di mercato per occupato',
       'reddito nazionale lordo ai prezzi di mercato per abitante',
       'redditi interni da lavoro dipendente per unità di lavoro dipendente',
       'redditi interni da lavoro dipendente per ora lavorata',
       'retribuzioni interne lorde per unità di lavoro dipendente',
       'retribuzioni interne lorde per ora lavorata',
       'retribuzioni interne lorde per occupato',
       'consumi finali nazionali per abitante'], dtype=object)

## Valori pro capite
- Dimensioni disponibili nel dataset: FREQ, REF_AREA, DATA_TYPE_AGGR, BRKDW_INDUSTRY_NACE_REV2, NONFIN_ASSETS, EXPEND_PURPOSE_COICOPCOFOG, VALUATION, ADJUSTMENT, PRICE, EDITION



### CONTI E AGGREGATI ECONOMICI TERRITORIALI	
- https://esploradati.istat.it/databrowser/#/it/dw/categories/IT1,DATAWAREHOUSE,1.0/UP_ACC_TERRIT/IT1,93_1227_DF_DCCN_TNA1_6,1.0
- https://esploradati.istat.it/SDMXWS/rest/dataflow/IT1/93_1227_DF_DCCN_TNA1_6/1.0/?detail=Full&references=Descendants
- https://esploradati.istat.it/SDMXWS/rest/data/IT1,93_1227_DF_DCCN_TNA1_6,1.0/A........./ALL/?detail=full&startPeriod=2014-01-01&endPeriod=2023-12-31&dimensionAtObservation=TIME_PERIOD
- Dimensioni disponibili nel dataset: FREQ, REF_AREA, DATA_TYPE_AGGR, BRKDW_INDUSTRY_NACE_REV2, NONFIN_ASSETS, EXPEND_PURPOSE_COICOPCOFOG, VALUATION, ADJUSTMENT, PRICE, EDITION

#### 93_1227

In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

In [9]:
dataset = '93_1227_DF_DCCN_TNA1_6' # territoriale
data_url = f'https://esploradati.istat.it/SDMXWS/rest/data/IT1,{dataset},1.0/A........./ALL/?detail=full&startPeriod=2014-01-01&endPeriod=2024-12-31&dimensionAtObservation=TIME_PERIOD'
response = requests.get(data_url)
root = ET.fromstring(response.content)
ns = {"message": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message","generic": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic"}
data = []
for series in root.findall(".//generic:Series", namespaces=ns):
    series_key = series.find("generic:SeriesKey", namespaces=ns)
    attributes = {value.attrib["id"]: value.attrib["value"] for value in series_key.findall("generic:Value", namespaces=ns)}
    for obs in series.findall("generic:Obs", namespaces=ns):
        time_period = obs.find("generic:ObsDimension", namespaces=ns).attrib["value"]
        obs_value = obs.find("generic:ObsValue", namespaces=ns).attrib["value"]
        data.append({**attributes, "TIME_PERIOD": time_period, "OBS_VALUE": float(obs_value)})
dati = pd.DataFrame(data)
# dati = dati.query('VALUATION == "V"')
dati = dati[['DATA_TYPE_AGGR', 'REF_AREA','EDITION','VALUATION', 'TIME_PERIOD', 'OBS_VALUE']]
dati = pd.merge(dati, TYPE_AGGR, how ='left', left_on = 'DATA_TYPE_AGGR', right_on = 'Codice')
dati = dati[['Descrizione','REF_AREA','Codice','EDITION','VALUATION','TIME_PERIOD','OBS_VALUE']]
dati = pd.merge(dati, CL_REFAREA, how ='left', left_on = 'REF_AREA', right_on = 'Istat')
dati = dati[['EDITION','Descrizione','REF_AREA','area','NUT','VALUATION','TIME_PERIOD','OBS_VALUE']]
# dati.to_sql(dataset, conn, if_exists="replace", index=False)
dati.shape

(9835, 8)

In [10]:
dati['Descrizione'].unique()

array(['valore aggiunto per unità di lavoro',
       'valore aggiunto per ora lavorata', 'valore aggiunto per abitante',
       'valore aggiunto per occupato',
       'prodotto interno lordo ai prezzi di mercato per abitante',
       'reddito disponibile delle famiglie consumarici per abitante',
       'redditi interni da lavoro dipendente per unità di lavoro dipendente',
       'redditi interni da lavoro dipendente per ora lavorata',
       'redditi interni da lavoro dipendente per occupato',
       'retribuzioni interne lorde per unità di lavoro dipendente',
       'retribuzioni interne lorde per ora lavorata',
       'retribuzioni interne lorde per occupato',
       'consumi finali interni per abitante'], dtype=object)

In [6]:
dati.query('REF_AREA == "ITF3"').sort_values(by =['Descrizione','TIME_PERIOD'], ascending=[True, False])

,EDITION,Descrizione,REF_AREA,area,NUT,VALUATION,TIME_PERIOD,OBS_VALUE
1301,2025M1,consumi finali interni per abitante,ITF3,Campania,ITF,V,2022,20496.079
1300,2025M1,consumi finali interni per abitante,ITF3,Campania,ITF,V,2021,18709.688
1281,2025M1,prodotto interno lordo ai prezzi di mercato pe...,ITF3,Campania,ITF,V,2023,23213.853
1283,2025M1,prodotto interno lordo ai prezzi di mercato pe...,ITF3,Campania,ITF,Y,2023,22046.048
1280,2025M1,prodotto interno lordo ai prezzi di mercato pe...,ITF3,Campania,ITF,V,2022,21718.615
1282,2025M1,prodotto interno lordo ai prezzi di mercato pe...,ITF3,Campania,ITF,Y,2022,21135.318
1279,2025M1,prodotto interno lordo ai prezzi di mercato pe...,ITF3,Campania,ITF,V,2021,19902.667
1293,2025M1,redditi interni da lavoro dipendente per occupato,ITF3,Campania,ITF,V,2023,34323.969
1292,2025M1,redditi interni da lavoro dipendente per occupato,ITF3,Campania,ITF,V,2022,33487.021
1291,2025M1,redditi interni da lavoro dipendente per occupato,ITF3,Campania,ITF,V,2021,32014.916


In [156]:
pd.pivot(dati, index=['EDITION','Descrizione','REF_AREA'], columns= 'TIME_PERIOD', values = 'OBS_VALUE')


TIME_PERIOD                                                 2021        2022  \
EDITION Descrizione                         REF_AREA                           
2025M1  consumi finali interni per abitante IT         23870.839   26473.745   
                                            ITC        26173.169   29097.918   
                                            ITC1       26013.592   28904.054   
                                            ITC2       37219.918   40873.352   
                                            ITC3       26749.890   29752.051   
...                                                          ...         ...   
        valore aggiunto per unità di lavoro ITFG       57179.488   59954.205   
                                            ITG        57505.896   61481.940   
                                            ITG1       58189.208   62228.421   
                                            ITG2       55811.449   59595.188   
                                            ITZ       231442.879  118461.510   

TIME_PERIOD                                           2023  
EDITION Descrizione                         REF_AREA        
2025M1  consumi finali interni per abitante IT         NaN  
                                            ITC        NaN  
                                            ITC1       NaN  
                                            ITC2       NaN  
                                            ITC3       NaN  
...                                                    ...  
        valore aggiunto per unità di lavoro ITFG       NaN  
                                            ITG        NaN  
                                            ITG1       NaN  
                                            ITG2       NaN  
                                            ITZ        NaN  

[737 rows x 3 columns]

In [125]:
dati['Descrizione'].unique()

array(['valore aggiunto per unità di lavoro',
       'valore aggiunto per ora lavorata', 'valore aggiunto per occupato',
       'prodotto interno lordo ai prezzi di mercato per unità di lavoro totali',
       'prodotto interno lordo ai prezzi di mercato per ora lavorata',
       'prodotto interno lordo ai prezzi di mercato per abitante',
       'prodotto interno lordo ai prezzi di mercato per occupato',
       'reddito nazionale lordo ai prezzi di mercato per abitante',
       'redditi interni da lavoro dipendente per unità di lavoro dipendente',
       'redditi interni da lavoro dipendente per ora lavorata',
       'retribuzioni interne lorde per unità di lavoro dipendente',
       'retribuzioni interne lorde per ora lavorata',
       'retribuzioni interne lorde per occupato',
       'consumi finali nazionali per abitante'], dtype=object)

In [9]:
print(dati['VALUATION'].unique())
print(dati['EDITION'].unique())


['V' 'Y']
['2025M1']


Ecco uno script completo in Python per estrarre i dati territoriali ISTAT dal dataset 93_1227_DF_DCCN_TNA1_6 (valori pro capite), utilizzando l’API REST SDMX di ISTAT. Lo script include la richiesta HTTP, il parsing XML e la conversione in un DataFrame pandas.

In [37]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# 📌 Parametri
dataset_id = '93_1227_DF_DCCN_TNA1_6'
base_url = 'https://esploradati.istat.it/SDMXWS/rest/data'
# Costruzione dell'URL
url = f"{base_url}/IT1,{dataset_id},1.0/ALL/ALL?startPeriod=2014&endPeriod=2023&detail=full"

# 📥 Richiesta HTTP
response = requests.get(url)
response.raise_for_status()  # Verifica che la richiesta sia andata a buon fine

# 🧩 Parsing XML
root = ET.fromstring(response.content)
ns = {
    'mes': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message',
    'gen': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic'
}

# 🔄 Estrazione dati
data = []
for series in root.findall('.//gen:Series', ns):
    series_key = series.find('gen:SeriesKey', ns)
    attributes = {value.attrib['id']: value.attrib['value'] for value in series_key.findall('gen:Value', ns)}
    for obs in series.findall('gen:Obs', ns):
        time_period = obs.find('gen:ObsDimension', ns).attrib['value']
        obs_value = obs.find('gen:ObsValue', ns).attrib['value']
        data.append({**attributes, 'TIME_PERIOD': time_period, 'OBS_VALUE': float(obs_value)})

# 📊 Creazione DataFrame
df = pd.DataFrame(data)

# 🔍 Filtraggio: ultima edizione e valori correnti
ultima_edizione = df['EDITION'].max()
df_filtrato = df[(df['VALUATION'] == 'V') & (df['EDITION'] == ultima_edizione)].copy()

# 💾 Salvataggio su CSV
df_filtrato.to_csv('dati_territoriali_istat.csv', index=False)

# ✅ Visualizzazione
(df_filtrato.head(20))


,FREQ,REF_AREA,DATA_TYPE_AGGR,BRKDW_INDUSTRY_NACE_REV2,NONFIN_ASSETS,EXPEND_PURPOSE_COICOPCOFOG,VALUATION,ADJUSTMENT,PRICE,EDITION,TIME_PERIOD,OBS_VALUE
0,A,IT,B1G_B_W2_S1_R_FT,Z,Z,Z,V,N,Z,2025M1,2021,71124.590
1,A,IT,B1G_B_W2_S1_R_FT,Z,Z,Z,V,N,Z,2025M1,2022,74816.000
2,A,IT,B1G_B_W2_S1_R_HW,Z,Z,Z,V,N,Z,2025M1,2021,39.672
3,A,IT,B1G_B_W2_S1_R_HW,Z,Z,Z,V,N,Z,2025M1,2022,41.477
4,A,IT,B1G_B_W2_S1_R_POP,Z,Z,Z,V,N,Z,2025M1,2021,27801.919
5,A,IT,B1G_B_W2_S1_R_POP,Z,Z,Z,V,N,Z,2025M1,2022,30375.724
6,A,IT,B1G_B_W2_S1_R_POP,Z,Z,Z,V,N,Z,2025M1,2023,32382.509
9,A,IT,B1G_B_W2_S1_R_PS,Z,Z,Z,V,N,Z,2025M1,2021,65578.611
10,A,IT,B1G_B_W2_S1_R_PS,Z,Z,Z,V,N,Z,2025M1,2022,70148.033
11,A,IT,B1G_B_W2_S1_R_PS,Z,Z,Z,V,N,Z,2025M1,2023,73379.038


In [36]:
import pandas as pd

# ⬇️ Assumiamo che `dati` sia il DataFrame iniziale
# Deve contenere: ['REF_AREA', 'area', 'NUT', 'OBS_VALUE', 'TIME_PERIOD', 'EDITION', 'VALUATION']

# 🔍 Filtro: ultima edizione disponibile e valori correnti
ultima_edizione = dati['EDITION'].max()
dati_filtrati = dati[
    (dati['VALUATION'] == 'V') &
    (dati['EDITION'] == ultima_edizione)
].copy()

# 🔢 Conversione valori numerici
dati_filtrati['OBS_VALUE'] = pd.to_numeric(dati_filtrati['OBS_VALUE'], errors='coerce')
dati_filtrati = dati_filtrati.dropna(subset=['OBS_VALUE'])

# 📊 Ordina i dati per regione e anno
dati_filtrati = dati_filtrati.sort_values(by=['REF_AREA', 'TIME_PERIOD'])

# 💾 Esporta in CSV
dati_filtrati.to_csv("dati_procapite_regioni.csv", index=False)

# ✅ Visualizza le prime righe
(dati_filtrati.head(20))


,EDITION,Descrizione,REF_AREA,area,NUT,VALUATION,TIME_PERIOD,OBS_VALUE
0,2025M1,valore aggiunto per unità di lavoro,IT,Italia,NaN,V,2021,71124.590
2,2025M1,valore aggiunto per ora lavorata,IT,Italia,NaN,V,2021,39.672
4,2025M1,valore aggiunto per abitante,IT,Italia,NaN,V,2021,27801.919
9,2025M1,valore aggiunto per occupato,IT,Italia,NaN,V,2021,65578.611
12,2025M1,prodotto interno lordo ai prezzi di mercato pe...,IT,Italia,NaN,V,2021,31158.595
17,2025M1,reddito disponibile delle famiglie consumarici...,IT,Italia,NaN,V,2021,20032.206
20,2025M1,redditi interni da lavoro dipendente per unità...,IT,Italia,NaN,V,2021,45110.283
22,2025M1,redditi interni da lavoro dipendente per ora l...,IT,Italia,NaN,V,2021,24.728
24,2025M1,redditi interni da lavoro dipendente per occupato,IT,Italia,NaN,V,2021,38553.615
27,2025M1,retribuzioni interne lorde per unità di lavoro...,IT,Italia,NaN,V,2021,33008.598


In [35]:
import pandas as pd

# ⬇️ Parte da un DataFrame già pronto chiamato `dati`
# Verifica: dati contiene colonne ['REF_AREA', 'OBS_VALUE', 'TIME_PERIOD', 'EDITION', 'VALUATION', 'area', 'NUT']

# 🔍 Filtra l'ultima edizione disponibile e valori correnti
ultima_edizione = dati['EDITION'].max()
dati_filtrati = dati[(dati['VALUATION'] == 'V') & (dati['EDITION'] == ultima_edizione)].copy()

# 🔢 Converte valori
dati_filtrati['OBS_VALUE'] = pd.to_numeric(dati_filtrati['OBS_VALUE'], errors='coerce')
dati_filtrati = dati_filtrati.dropna(subset=['OBS_VALUE'])

# 🧱 Calcola indice base 100 (base = primo anno disponibile per ogni regione)
anno_base = dati_filtrati['TIME_PERIOD'].min()
base = dati_filtrati[dati_filtrati['TIME_PERIOD'] == anno_base].set_index('REF_AREA')['OBS_VALUE'].to_dict()

# Aggiunge la base a ogni riga
dati_filtrati['BASE'] = dati_filtrati['REF_AREA'].map(base)
dati_filtrati = dati_filtrati[dati_filtrati['BASE'].notna()]
dati_filtrati['INDICE'] = dati_filtrati['OBS_VALUE'] / dati_filtrati['BASE'] * 100

# 📊 Calcola indice sintetico medio per ciascuna regione
indice_sintetico = (
    dati_filtrati
    .groupby(['REF_AREA', 'area', 'NUT'])['INDICE']
    .mean()
    .reset_index()
    .rename(columns={'INDICE': 'INDICE_MEDIA_2014_2023'})
    .sort_values(by='INDICE_MEDIA_2014_2023', ascending=False)
)

# 💾 Salva in CSV
indice_sintetico.to_csv("indice_sintetico_regioni.csv", index=False)

# ✅ Visualizza
print(indice_sintetico)


    REF_AREA                            area   NUT  INDICE_MEDIA_2014_2023
136      ITZ                     Extra-Regio    IT                     inf
130    ITG19                        Siracusa  ITG1              189.913269
127    ITG16                            Enna  ITG1              178.494510
125    ITG14                       Agrigento  ITG1              173.834752
126    ITG15                   Caltanissetta  ITG1              172.578772
..       ...                             ...   ...                     ...
95      ITF2                          Molise   ITF              116.849767
25     ITC45                          Milano  ITC4              114.883589
131     ITG2                        Sardegna   ITG              114.851379
113     ITF6                        Calabria   ITF              109.780327
13      ITC2  Valle d'Aosta / Vallée d'Aoste   ITC               96.643282

[137 rows x 4 columns]


## [93_498 CONTI E AGGREGATI ECONOMICI TERRITORIALI|Prodotto interno lordo lato produzione	](https://esploradati.istat.it/SDMXWS/rest/data/IT1,93_498_DF_DCCN_PILT_1,1.0/ALL/ALL?detail=full)

In [33]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# === PARAMETRI CONFIGURABILI ===
dataset_id = '93_498_DF_DCCN_PILT_1'
base_url = 'https://esploradati.istat.it/SDMXWS/rest/data'
start_year = 2014
end_year = 2025

# SDMX key structure: /IT1,<DATAFLOW_ID>,1.0/ALL/ALL
url = f"{base_url}/IT1,{dataset_id},1.0/ALL/ALL?startPeriod={start_year}-01-01&endPeriod={end_year}-12-31&detail=full"

# === RICHIESTA DATI ===
response = requests.get(url)
response.raise_for_status()

# === PARSING XML SDMX ===
ns = {
    'mes': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message',
    'gen': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic'
}
root = ET.fromstring(response.content)

data = []
for series in root.findall('.//gen:Series', ns):
    series_key = series.find('gen:SeriesKey', ns)
    attributes = {value.attrib['id']: value.attrib['value'] for value in series_key.findall('gen:Value', ns)}
    for obs in series.findall('gen:Obs', ns):
        time_period = obs.find('gen:ObsDimension', ns).attrib['value']
        obs_value = obs.find('gen:ObsValue', ns).attrib.get('value')
        if obs_value:
            data.append({**attributes, 'TIME_PERIOD': time_period, 'OBS_VALUE': float(obs_value)})

# === CREAZIONE DATAFRAME ===
df = pd.DataFrame(data)

# === FILTRO: ultima edizione + valori a prezzi correnti ("V") ===
ultima_edizione = df['EDITION'].max()
df_filtrato = df[
    (df['VALUATION'] == 'V') & 
    (df['EDITION'] == ultima_edizione)
].copy()

# === SALVATAGGIO FILE ===
df_filtrato.to_csv('pil_territoriale_istat.csv', index=False)

# === VISUALIZZAZIONE RISULTATI ===
df_filtrato.head(20)


,FREQ,REF_AREA,DATA_TYPE_AGGR,VALUATION,ADJUSTMENT,EDITION,TIME_PERIOD,OBS_VALUE
0,A,IT,B1G_B_W2_S1,V,N,2025M1,2021,1644016.4
1,A,IT,B1G_B_W2_S1,V,N,2025M1,2022,1792583.9
2,A,IT,B1G_B_W2_S1,V,N,2025M1,2023,1910056.4
5,A,IT,B1GQ_B_W2_S1,V,N,2025M1,2021,1842507.4
6,A,IT,B1GQ_B_W2_S1,V,N,2025M1,2022,1997054.9
7,A,IT,B1GQ_B_W2_S1,V,N,2025M1,2023,2128001.4
10,A,IT,D21X31_C_W2_S1,V,N,2025M1,2021,198491.0
11,A,IT,D21X31_C_W2_S1,V,N,2025M1,2022,204471.0
12,A,IT,D21X31_C_W2_S1,V,N,2025M1,2023,217945.0
15,A,IT108,B1G_B_W2_S1,V,N,2025M1,2021,26212.0


In [34]:
print("Valori unici in EDITION:", df['EDITION'].unique())

Valori unici in EDITION: ['2025M1']


In [32]:
print("Valori unici in TIME_PERIOD:", df['TIME_PERIOD'].unique())

Valori unici in TIME_PERIOD: ['2021' '2022' '2023']


In [27]:
print("Valori unici in DATA_TYPE_AGGR:", df['DATA_TYPE_AGGR'].unique())


Valori unici in DATA_TYPE_AGGR: ['B1G_B_W2_S1' 'B1GQ_B_W2_S1' 'D21X31_C_W2_S1']


| Codice           | Significato                                                                                                     |
| ---------------- | --------------------------------------------------------------------------------------------------------------- |
| `B1G_B_W2_S1`    | **Valore aggiunto lordo** ai **prezzi base (B)**, **prezzi concatenati (W2)**, settore **S1** (Totale economia) |
| `B1GQ_B_W2_S1`   | **Prodotto interno lordo (PIL)** ai **prezzi di mercato (B)**, **prezzi concatenati (W2)**, settore **S1**      |
| `D21X31_C_W2_S1` | **Imposte nette sulla produzione** ai **prezzi correnti (C)**, concatenati **(W2)**, settore **S1**             |


Nel linguaggio SDMX (e secondo la classificazione **SEC 2010**, usata anche da ISTAT), il codice **`S1`** nella dimensione **`SETTORE`** (o `SECTOR`) indica:


| Codice  | Nome settore                                                | Descrizione sintetica                                                   |
| ------- | ----------------------------------------------------------- | ----------------------------------------------------------------------- |
| **S1**  | Totale economia                                             | Tutti i settori istituzionali (somma di S11–S15)                        |
| **S11** | Società non finanziarie                                     | Imprese private e pubbliche che producono beni/servizi non finanziari   |
| **S12** | Società finanziarie                                         | Banche, assicurazioni, fondi, società di leasing, ecc.                  |
| **S13** | Amministrazioni pubbliche                                   | Stato, regioni, comuni, INPS, enti pubblici non commerciali             |
| **S14** | Famiglie                                                    | Consumatori, imprenditori individuali, lavoratori autonomi              |
| **S15** | Istituzioni senza scopo di lucro al servizio delle famiglie | Fondazioni, associazioni, ONG, enti religiosi, partiti politici privati |

B1GQ_B_W2_S1 = PIL territoriale, ai prezzi di mercato, espresso a prezzi concatenati, riferito alla totalità dell’economia (S1).

❓Ma... perché ci sono sia B che W2? Questo è un codice tecnico SDMX che unisce:

tipo di prezzo (B, C, K...)

tipo di valutazione reale o nominale (W1, W2 per prezzi concatenati o volumi reali)

Nel contesto ISTAT, il significato pratico è:

B → valore monetario

W2 → è a prezzi concatenati → volume reale

👉 quindi non è una doppia valutazione, ma una combinazione SDMX formale.

✅ Conclusione
✔️ B1GQ_B_W2_S1 è PIL reale (a volume) riferito a tutta l’economia territoriale, espresso a prezzi concatenati, ovvero depurato dall’inflazione.



In [30]:
SETTORI_ISTITUZIONALI = {
    'S1':  'Totale economia',
    'S11': 'Società non finanziarie',
    'S12': 'Società finanziarie',
    'S13': 'Amministrazioni pubbliche',
    'S14': 'Famiglie',
    'S15': 'Istituzioni senza scopo di lucro al servizio delle famiglie (ISPFL)'
}

## dizionario per Conti economici (DATA_TYPE_AGGR)

AGGREGATI_CONTABILI = {
    'B1GQ': 'PIL ai prezzi di mercato',
    'B1G':  'Valore aggiunto lordo ai prezzi base',
    'D1':   'Remunerazione dei dipendenti',
    'D21':  'Imposte sulla produzione',
    'D31':  'Sussidi alla produzione',
    'D21X31': 'Imposte nette sulla produzione',
    'P1':   'Produzione',
    'P2':   'Consumi intermedi',
    'P3':   'Spesa per consumi finali',
    'P5G':  'Formazione lorda di capitale',
    'P51G': 'Investimenti fissi lordi',
    'K1':   'Stock di capitale',
    'B2A3G': 'Risultato lordo di gestione + reddito misto',
    'B5G':  'Reddito nazionale lordo',
    'B6G':  'Reddito disponibile lordo',
    'B8G':  'Risparmio lordo'
}


In [31]:
import pandas as pd

# === FUNZIONE DI DECODIFICA ===
def decodifica_codice_sdmx(codice):
    try:
        aggregato, prezzi, valutazione, settore = codice.split("_")
    except ValueError:
        return {
            "codice": codice,
            "aggregato": None,
            "prezzi": None,
            "valutazione": None,
            "settore": None
        }

    AGGREGATI = {
        'B1GQ': 'PIL ai prezzi di mercato',
        'B1G': 'Valore aggiunto lordo',
        'D1': 'Remunerazione dei dipendenti',
        'D21': 'Imposte sulla produzione',
        'D31': 'Sussidi alla produzione',
        'D21X31': 'Imposte nette sulla produzione',
        'P1': 'Produzione totale',
        'P2': 'Consumi intermedi',
        'P3': 'Spesa per consumi finali',
        'P51G': 'Investimenti fissi lordi',
        'K1': 'Stock di capitale',
        'B5G': 'Reddito nazionale lordo'
    }

    PREZZI = {
        'B': 'Prezzi correnti',
        'C': 'Prezzi correnti (dettaglio)',
        'K': 'Prezzi costanti',
        'X': 'Prezzi base',
    }

    VALUTAZIONI = {
        'W1': 'Prezzi costanti (volume)',
        'W2': 'Prezzi concatenati (volume reale)',
        'W3': 'Prezzi del periodo precedente',
    }

    SETTORI = {
        'S1': 'Totale economia',
        'S11': 'Società non finanziarie',
        'S12': 'Società finanziarie',
        'S13': 'Amministrazioni pubbliche',
        'S14': 'Famiglie',
        'S15': 'ISPFL'
    }

    return {
        "codice": codice,
        "aggregato": AGGREGATI.get(aggregato, aggregato),
        "prezzi": PREZZI.get(prezzi, prezzi),
        "valutazione": VALUTAZIONI.get(valutazione, valutazione),
        "settore": SETTORI.get(settore, settore)
    }

# === APPLICA LA DECODIFICA AL DATAFRAME ===
df_decodificato = df['DATA_TYPE_AGGR'].apply(decodifica_codice_sdmx).apply(pd.Series)

# === UNISCE LE NUOVE COLONNE AL DATAFRAME ORIGINALE ===
df = pd.concat([df, df_decodificato[['aggregato', 'prezzi', 'valutazione', 'settore']]], axis=1)

# === SALVA IN CSV ===
df.to_csv('dati_decodificati_sdmx.csv', index=False)

# ✅ VISUALIZZA ANTEPRIMA
print(df[['DATA_TYPE_AGGR', 'aggregato', 'prezzi', 'valutazione', 'settore']].drop_duplicates())


    DATA_TYPE_AGGR  ...          settore
0      B1G_B_W2_S1  ...  Totale economia
5     B1GQ_B_W2_S1  ...  Totale economia
10  D21X31_C_W2_S1  ...  Totale economia

[3 rows x 5 columns]


## 93_1227_DF_DCCN_TNA1_1 Valore aggiunto per branca di attività

In [ ]:
# 
dataset = '93_1227_DF_DCCN_TNA1_1'
data_url = f'https://esploradati.istat.it/SDMXWS/rest/data/IT1,{dataset},1.0/A........./ALL/?detail=full&startPeriod=2014-01-01&endPeriod=2023-12-31&dimensionAtObservation=TIME_PERIOD'
response = requests.get(data_url)
root = ET.fromstring(response.content)
# 🔎 Namespace XML per SDMX aggiornato
ns = {"message": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message","generic": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic"}
data = []
for series in root.findall(".//generic:Series", namespaces=ns):
    series_key = series.find("generic:SeriesKey", namespaces=ns)
    attributes = {value.attrib["id"]: value.attrib["value"] for value in series_key.findall("generic:Value", namespaces=ns)}
    for obs in series.findall("generic:Obs", namespaces=ns):
        time_period = obs.find("generic:ObsDimension", namespaces=ns).attrib["value"]
        obs_value = obs.find("generic:ObsValue", namespaces=ns).attrib["value"]
        data.append({**attributes, "TIME_PERIOD": time_period, "OBS_VALUE": float(obs_value)})
df = pd.DataFrame(data)
df.to_sql(dataset, conn, if_exists="replace", index=False)
df.shape

(8246, 12)

In [ ]:
# 115_168 - Produzione industriale per settore economico (Ateco 2 cifre)
dataset = '115_168_DF_DCSP_PRODCOM_1'
data_url = f"{root}{dataset},1.0/A.../ALL/?detail=full&dimensionAtObservation=TIME_PERIOD"
response = requests.get(data_url)
root = ET.fromstring(response.content)
# 🔎 Namespace XML per SDMX aggiornato
ns = {"message": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message","generic": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic"}
data = []
for series in root.findall(".//generic:Series", namespaces=ns):
    series_key = series.find("generic:SeriesKey", namespaces=ns)
    attributes = {value.attrib["id"]: value.attrib["value"] for value in series_key.findall("generic:Value", namespaces=ns)}
    for obs in series.findall("generic:Obs", namespaces=ns):
        time_period = obs.find("generic:ObsDimension", namespaces=ns).attrib["value"]
        obs_value = obs.find("generic:ObsValue", namespaces=ns).attrib["value"]
        data.append({**attributes, "TIME_PERIOD": time_period, "OBS_VALUE": float(obs_value)})
df = pd.DataFrame(data)
df.to_sql(dataset, conn, if_exists="replace", index=False)
df.shape

(304, 6)

In [ ]:
# 163_88  CONTI E AGGREGATI ECONOMICI NAZIONALI TRIMESTRALI - Occupazione per branca di attività	
dataset = '163_88_DF_DCCN_OCCQSEC2010_1'
data_url = f"https://esploradati.istat.it/SDMXWS/rest/data/IT1,{dataset},1.0/Q......../ALL/?detail=full&startPeriod=2022-06-01&endPeriod=2024-12-31&dimensionAtObservation=TIME_PERIOD"
response = requests.get(data_url)
root = ET.fromstring(response.content)
ns = {"message": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message","generic": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic"}
data = []
for series in root.findall(".//generic:Series", namespaces=ns):
    series_key = series.find("generic:SeriesKey", namespaces=ns)
    attributes = {value.attrib["id"]: value.attrib["value"] for value in series_key.findall("generic:Value", namespaces=ns)}
    for obs in series.findall("generic:Obs", namespaces=ns):
        time_period = obs.find("generic:ObsDimension", namespaces=ns).attrib.get("value", "N/A")
        obs_value_node = obs.find("generic:ObsValue", namespaces=ns)
        # Verifica se il valore dell'osservazione esiste
        if obs_value_node is not None and "value" in obs_value_node.attrib:
            try:
                obs_value = float(obs_value_node.attrib["value"])
            except ValueError:
                obs_value = None  # Gestione valori non numerici
        else:
            obs_value = None  # Se il valore manca
        data.append({**attributes, "TIME_PERIOD": time_period, "OBS_VALUE": obs_value})
df = pd.DataFrame(data)
df.to_sql(dataset, conn, if_exists="replace", index=False)
df.shape

(16767, 11)

In [52]:
# IMPRESE/Struttura/Imprese e addetti/Forma giuridica, settori economici (Ateco 5 cifre) - Italia 
dataset = '183_277_DF_DICA_ASIAUE1P_2'
data_url = f'https://esploradati.istat.it/SDMXWS/rest/data/IT1,{dataset},1.0/A......./ALL/?detail=full&startPeriod=2022-01-01&endPeriod=2022-12-31&dimensionAtObservation=TIME_PERIOD'

response = requests.get(data_url)
root = ET.fromstring(response.content)
ns = {"message": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message","generic": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic"}
data = []
for series in root.findall(".//generic:Series", namespaces=ns):
    series_key = series.find("generic:SeriesKey", namespaces=ns)
    attributes = {value.attrib["id"]: value.attrib["value"] for value in series_key.findall("generic:Value", namespaces=ns)}
    for obs in series.findall("generic:Obs", namespaces=ns):
        time_period = obs.find("generic:ObsDimension", namespaces=ns).attrib.get("value", "N/A")
        obs_value_node = obs.find("generic:ObsValue", namespaces=ns)
        # Verifica se il valore dell'osservazione esiste
        if obs_value_node is not None and "value" in obs_value_node.attrib:
            try:
                obs_value = float(obs_value_node.attrib["value"])
            except ValueError:
                obs_value = None  # Gestione valori non numerici
        else:
            obs_value = None  # Se il valore manca
        data.append({**attributes, "TIME_PERIOD": time_period, "OBS_VALUE": obs_value})
df = pd.DataFrame(data)
df.to_sql(dataset, conn, if_exists="replace", index=False)
df.shape

(79896, 10)

In [30]:
# IMPRESE/Struttura/Imprese e addetti/Forma giuridica, settori economici (Ateco 3 cifre) - prov.
dataset = '183_277_DF_DICA_ASIAUE1P_4'
data_url = f'https://esploradati.istat.it/SDMXWS/rest/data/IT1,{dataset},1.0/A......./ALL/?detail=full&startPeriod=2022-01-01&endPeriod=2022-12-31&dimensionAtObservation=TIME_PERIOD'

response = requests.get(data_url)
root = ET.fromstring(response.content)
ns = {"message": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message","generic": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic"}
data = []
for series in root.findall(".//generic:Series", namespaces=ns):
    series_key = series.find("generic:SeriesKey", namespaces=ns)
    attributes = {value.attrib["id"]: value.attrib["value"] for value in series_key.findall("generic:Value", namespaces=ns)}
    for obs in series.findall("generic:Obs", namespaces=ns):
        time_period = obs.find("generic:ObsDimension", namespaces=ns).attrib.get("value", "N/A")
        obs_value_node = obs.find("generic:ObsValue", namespaces=ns)
        # Verifica se il valore dell'osservazione esiste
        if obs_value_node is not None and "value" in obs_value_node.attrib:
            try:
                obs_value = float(obs_value_node.attrib["value"])
            except ValueError:
                obs_value = None  # Gestione valori non numerici
        else:
            obs_value = None  # Se il valore manca
        data.append({**attributes, "TIME_PERIOD": time_period, "OBS_VALUE": obs_value})
df = pd.DataFrame(data)
df.to_sql(dataset, conn, if_exists="replace", index=False)
df.shape

(1263590, 10)

In [ ]:
# IMPRESE/Struttura/Imprese e addetti/Classe di addetti, settori economici (Ateco 2 cifre) - prov.
# https://esploradati.istat.it/databrowser/#/it/dw/categories/IT1,Z0900ENT,1.0/ENT_STRU/DICA_ASIAUE1P/IT1,183_277_DF_DICA_ASIAUE1P_5,1.0
# https://esploradati.istat.it/SDMXWS/rest/dataflow/IT1/183_277_DF_DICA_ASIAUE1P_5/1.0/?detail=Full&references=Descendants
# https://esploradati.istat.it/SDMXWS/rest/data/IT1,183_277_DF_DICA_ASIAUE1P_5,1.0/A......./ALL/?detail=full&dimensionAtObservation=TIME_PERIOD
dataset = '183_277_DF_DICA_ASIAUE1P_5'
data_url ='https://esploradati.istat.it/SDMXWS/rest/data/IT1,183_277_DF_DICA_ASIAUE1P_5,1.0/A......./ALL/?detail=full&dimensionAtObservation=TIME_PERIOD'
response = requests.get(data_url)
root = ET.fromstring(response.content)
ns = {"message": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message","generic": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic"}
data = []
for series in root.findall(".//generic:Series", namespaces=ns):
    series_key = series.find("generic:SeriesKey", namespaces=ns)
    attributes = {value.attrib["id"]: value.attrib["value"] for value in series_key.findall("generic:Value", namespaces=ns)}
    for obs in series.findall("generic:Obs", namespaces=ns):
        time_period = obs.find("generic:ObsDimension", namespaces=ns).attrib.get("value", "N/A")
        obs_value_node = obs.find("generic:ObsValue", namespaces=ns)
        # Verifica se il valore dell'osservazione esiste
        if obs_value_node is not None and "value" in obs_value_node.attrib:
            try:
                obs_value = float(obs_value_node.attrib["value"])
            except ValueError:
                obs_value = None  # Gestione valori non numerici
        else:
            obs_value = None  # Se il valore manca
        data.append({**attributes, "TIME_PERIOD": time_period, "OBS_VALUE": obs_value})
df = pd.DataFrame(data)
df.to_sql(dataset, conn, if_exists="replace", index=False)
df.shape

ParseError: syntax error: line 1, column 0 (<string>)

In [36]:
import requests
import xml.etree.ElementTree as ET

# URL SDMX ISTAT (correggi la query se necessario)
data_url = 'https://esploradati.istat.it/SDMXWS/rest/data/IT1,183_277_DF_DICA_ASIAUE1P_5,1.0/A......./ALL/?detail=full&dimensionAtObservation=TIME_PERIOD'

# Effettua la richiesta HTTP
response = requests.get(data_url)

# Debug: Controlla se la risposta è vuota o contiene errori
if response.status_code != 200:
    print(f"Errore HTTP {response.status_code}: {response.text}")
elif not response.content:
    print("⚠️ La risposta è vuota. Verifica l'URL.")
else:
    try:
        # Parsing dell'XML solo se la risposta è valida
        root = ET.fromstring(response.content)
        print("✅ XML caricato correttamente!")
    except ET.ParseError:
        print("❌ Errore nel parsing dell'XML. La risposta potrebbe non essere in formato SDMX.")
        print("🔍 Contenuto della risposta:", response.text[:500])  # Mostra i primi 500 caratteri


Errore HTTP 404: Error while retrieving Mappings from "Mapping Store"! Cause:Dataflow 'urn:sdmx:org.sdmx.infomodel.datastructure.Dataflow=IT1:183_277_DF_DICA_ASIAUE1P_5(1.0)' doesn't contain a mapping set


## Backup

In [139]:
# struttura = '92_1225_DF_DCCN_ANA1_7'
struttura = '93_1227_DF_DCCN_TNA1_6'
url = f"https://esploradati.istat.it/SDMXWS/rest/dataflow/IT1/{struttura}/1.0/?detail=Full&references=Descendants"
response = requests.get(url)
root = ET.fromstring(response.content)
ns = {"sdmx": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/structure"} # 🔎 Namespace SDMX
dimensions = root.findall(".//sdmx:DimensionList/sdmx:Dimension", namespaces=ns) # 📋
print("📊 Dimensioni disponibili nel dataset:") # Stampa le dimensioni trovate
for dim in dimensions:
    print(f"- ID: {dim.attrib['id']}")


📊 Dimensioni disponibili nel dataset:
- ID: FREQ
- ID: REF_AREA
- ID: DATA_TYPE_AGGR
- ID: BRKDW_INDUSTRY_NACE_REV2
- ID: NONFIN_ASSETS
- ID: EXPEND_PURPOSE_COICOPCOFOG
- ID: VALUATION
- ID: ADJUSTMENT
- ID: PRICE
- ID: EDITION


In [ ]:
root = ET.fromstring(response.content)
ns = {
    "message": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message",
    "structure": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/structure",
    "common": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common"
}
codelists = root.findall(".//structure:Codelist", namespaces=ns)
data = []
for codelist in codelists:
    codelist_id = codelist.attrib['id']
    codes = codelist.findall("structure:Code", namespaces=ns)
    
    for code in codes:
        code_id = code.attrib['id']
        label_text = "N/A"

        # Priorità alla descrizione in italiano
        for name in code.findall("common:Name", namespaces=ns):
            if name.attrib.get("{http://www.w3.org/XML/1998/namespace}lang") == "it":
                label_text = name.text
                break
            elif label_text == "N/A":  # Se manca l'italiano, prendi il primo disponibile
                label_text = name.text

        data.append((codelist_id, code_id, label_text))
struttura = pd.DataFrame(data, columns=["Codelist", "Valore", "Descrizione"])
struttura

,Codelist,Valore,Descrizione
0,CL_BASE_YEAR,0000,non specificato
1,CL_BASE_YEAR,1980,anno 1980
2,CL_BASE_YEAR,1981,anno 1981
3,CL_BASE_YEAR,1982,anno 1982
4,CL_BASE_YEAR,1989,anno 1989
...,...,...,...
16519,CL_VAL,E,euro per abitante
16520,CL_VAL,Z,non applicabile
16521,CL_VAL,G1,variazione congiunturale
16522,CL_VAL,G4,variazione tendenziale


In [136]:
struttura['Codelist'].unique()

array(['CL_BASE_YEAR', 'CL_CONTAB_NOTE', 'CL_CORREZ', 'CL_DS_NOTE',
       'CL_FLAG', 'CL_FREQ', 'CL_ITTER107', 'CL_TIPO_DATO_PIL_SEC2010',
       'CL_T_BIS', 'CL_UNIT_MEASURE', 'CL_UNIT_MULT', 'CL_VAL'],
      dtype=object)

## buttare

In [44]:
import pandasdmx

# === CREAZIONE CLIENT ===
estat = pandasdmx.Request('ESTAT')  # Eurostat

# === VISUALIZZA DATAFLOW DISPONIBILI ===
resp = estat.dataflow()
for key in list(resp.dataflow.keys())[:5]:  # primi 5 dataset
    print(key)

# === SCARICA UN DATASET: PIL nominale per nazione ===
# Dataset: nama_10_gdp (GDP and main components)
resp = estat.data(
    resource_id='nama_10_gdp',
    key={'geo': 'IT', 'na_item': 'B1GQ', 'unit': 'CLV10_MEUR', 's_adj': 'NSA'},
    params={'startPeriod': '2010', 'endPeriod': '2023'}
)

# === CONVERTI IN DATAFRAME ===
df = resp.to_pandas()
print(df.tail())




LFST_DPW_05
LFST_DPW_06
LFST_DPW_07
LFST_DPW_08
LFST_DPW_09


KeyError: 'nama_10_gdp'

In [45]:
resp

<pandasdmx.StructureMessage>
  <Header>
    id: 'DF1747256413'
    prepared: '2025-05-14T23:00:13.772000+02:00'
    sender: <Agency ESTAT>
    source: 
    test: False
  response: <Response [200]>
  DataflowDefinition (7935): LFST_DPW_05 LFST_DPW_06 LFST_DPW_07 LFST_D...
  DataStructureDefinition (7486): LFST_DPW_05 LFST_DPW_06 LFST_DPW_07 L...